In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import sys
import os
import torch
import time

from models.resnet import resnet18, resnet34, resnet50

In [ ]:
# CIFAR-100 데이터셋 로드
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=16)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18().to(device)  

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, trainloader, criterion, optimizer, device, epoch):
    """
    Function for model training    
    Args:
        model: The model to train
        trainloader: Training data loader
        criterion: Loss function
        optimizer: Optimizer
        device: Device to use for training (cuda or cpu)
        epoch: Current epoch number
        
    Returns:
        epoch_loss: Average loss for the epoch    """
    model.train()   # Set model to training mode
    start_time = time.time()  # Start time measurement
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(trainloader)
    
    train_time = time.time() - start_time
    
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Time: {train_time:.2f}s')
    
    return epoch_loss

In [ ]:
def evaluate(model, testloader, criterion, device):
    """
    모델 평가 함수
    
    Args:
        model: The model to train
        trainloader: Training data loader
        criterion: Loss function
        optimizer: Optimizer
        device: Device to use for training (cuda or cpu)
        epoch: Current epoch number
        
    Returns:
        epoch_loss: Average loss for the epoch
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    test_loss = 0.0
    correct = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # 평균 손실 및 정확도 계산
    test_loss = test_loss / len(testloader)
    accuracy = 100.0 * correct / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%, Time: {eval_time:.2f}s')
    
    return test_loss, accuracy

In [ ]:
# 메인 학습 루프 예시
def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs=200):
    """
    메인 학습 루프
    
    Args:
        model: 학습할 모델
        trainloader: 학습 데이터 로더
        testloader: 테스트 데이터 로더
        criterion: 손실 함수
        optimizer: 옵티마이저
        device: 학습에 사용할 디바이스 (cuda 또는 cpu)
        num_epochs: 학습할 에폭 수
    """
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        # 학습
        train_loss = train(model, trainloader, criterion, optimizer, device, epoch)
        
        # 평가 (필요한 경우)
        if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:  # 10 에폭마다 또는 마지막 에폭에 평가
            test_loss, accuracy = evaluate(model, testloader, criterion, device)
            
            # 최고 정확도 모델 저장
            if accuracy > best_acc:
                best_acc = accuracy
                print(f'새로운 최고 정확도: {best_acc:.2f}%')
                torch.save(model.state_dict(), 'best_model.pth')
    
    print(f'학습 완료! 최고 정확도: {best_acc:.2f}%')